In [ ]:
import pandas as pd
import openpyxl
import warnings
warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)

In [ ]:
df = pd.read_excel('otomoto_v2.xlsx')

In [ ]:
#pre check
df.info()

In [ ]:
#filtering out
df = df[~df['Link'].str.contains('https://carsmile.pl')]
df = df.dropna(subset=['Cena']) #invalid ads

In [ ]:
#irrelevant data
cols_to_drop = ['Homologacja ciężarowa','Spalanie W Cyklu Mieszanym','Wartość wykupu','Liczba pozostałych rat','Opłata początkowa','Miesięczna rata','lub do (przebieg km)','Okres gwarancji producenta',
                'VAT marża','Gwarancja dealerska (w cenie)','Możliwość finansowania','Numer rejestracyjny pojazdu','Spalanie Poza Miastem','Emisja CO2','Spalanie W Mieście']
df.drop(cols_to_drop, axis=1, inplace= True)

In [ ]:
#no if not mentioned
true_false_cols = ['Uszkodzony','Leasing','Tuning',' Pierwszy właściciel (od nowości)','Serwisowany w ASO','Zarejestrowany w Polsce','Faktura VAT','Ma numer rejestracyjny','Pokaż oferty z numerem VIN',
                   'Bezwypadkowy','Kierownica po prawej (Anglik)']
df[true_false_cols] = df[true_false_cols].fillna('Nie')

In [ ]:
#not specified
df[['Wersja','Rodzaj koloru','Kraj pochodzenia','Skrzynia biegów']] = df[['Wersja','Rodzaj koloru','Kraj pochodzenia','Skrzynia biegów']].fillna('Nie podano')

In [ ]:
#generation based on production year
df['Generacja'] = df[['Rok produkcji','Generacja']].apply(lambda x: '8T (2007-2016)' if x['Rok produkcji'] <= 2015 else x['Generacja'] if x['Rok produkcji'] == 2016 else 'F5 (2016-)', axis=1)
#as 8T gen is majority
df['Generacja'] = df['Generacja'].fillna('8T (2007-2016)')

In [ ]:
def mileage(x):
    try:
        return int(x.replace('km','').replace(' ',''))
    except:
        return None

In [ ]:
#filling missing mileage with mean value for used cars

df['Przebieg'] = df['Przebieg'].map(mileage)
df['Przebieg'] = df[['Przebieg','Rok produkcji']].apply(lambda x: 0 if str(x['Przebieg']) == 'nan' and x['Rok produkcji'] == 2023 else x['Przebieg'], axis=1)
mean_val = int(df['Przebieg'].mean())
df['Przebieg'] = df['Przebieg'].fillna(mean_val)

In [ ]:
#based on technical knowledge
df['Napęd'] = df['Napęd'].replace('4x4 (dołączany automatycznie)','4x4 (stały)').replace('4x4 (dołączany ręcznie)','4x4 (stały)').replace('Na tylne koła','Nie określono').fillna('Nie określono')

In [ ]:
#there were only 5 and 2 doors wersions
df['Liczba drzwi'] = df['Liczba drzwi'].fillna(5).map(lambda x: 5 if x >= 4 else 2)

In [ ]:
def body_type(type,doors):
    if type == 'Kabriolet':
        doors = 2
    elif doors == 5:
        type = 'Sedan'
    elif doors == 2:
        type = 'Coupe'
    return type

In [ ]:
#body type base on doors number and oryginal body type value (cabrio)
df['Typ nadwozia'] = df[['Typ nadwozia','Liczba drzwi']].apply(lambda x: body_type(x['Typ nadwozia'],x['Liczba drzwi']) , axis=1)

In [ ]:
#seats number base on body type
df['Liczba miejsc'] = df['Typ nadwozia'].map({'Kabriolet': 4, 'Coupe': 4, 'Sedan':5})

In [ ]:
#first registration date same as production year if not mentioned
df['Data pierwszej rejestracji w historii pojazdu'] = df[['Data pierwszej rejestracji w historii pojazdu','Rok produkcji']].apply(
    lambda x: 'styczeń ' + str(x['Rok produkcji']) if str(x['Data pierwszej rejestracji w historii pojazdu']) == 'nan' else x['Data pierwszej rejestracji w historii pojazdu'], axis=1)

In [ ]:
#data formats
df['Cena'] = df['Cena'].str.replace(',','.').str.replace(' ','').astype('float').astype('int')
df['Przebieg'] = df['Przebieg'].astype('int')
df['Pojemność skokowa'] = df['Pojemność skokowa'].str.replace(' ', '').str.replace('cm3', '').astype('int')
df['Moc'] = df['Moc'].str.replace(' KM', '').astype('int')

In [ ]:
#post check
df.info()

In [ ]:
#(Yes, No) translation
for col in df.columns:
    if str(df[col].unique()) == "['Nie' 'Tak']" or str(df[col].unique()) == "['Tak' 'Nie']":
        df[col] = df[col].str.replace('Tak','Yes').replace('Nie','No')

In [ ]:
df.to_excel('otomoto_v2_data_cleaning.xlsx', index = False)